In [1]:
import sympy
from einsteinpy.symbolic import MetricTensor

#import inflatox
import os, sys
inflatox_mod_path = os.path.abspath(os.path.join('./python/src/'))
if inflatox_mod_path not in sys.path:
  sys.path.append(inflatox_mod_path)
print(sys.path)
from inflatox import SymbolicCalculation, Compiler

['/home/raulwolters/Documents/inflatox', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/raulwolters/Documents/python310env/lib64/python3.10/site-packages', '/home/raulwolters/Documents/python310env/lib/python3.10/site-packages', '/home/raulwolters/Documents/inflatox/python/src']


In [2]:
sympy.init_printing()
φ, θ, ψ = sympy.symbols('φ θ ψ')
d = 3
coords = [φ, θ, ψ]

g = [[0 for _ in range(d)] for _ in range(d)]
g[0][0] = 1
g[1][1] = φ**2
g[2][2] = (φ**2) * (sympy.sin(θ)**2)
g = MetricTensor(g ,(φ, θ, ψ))
display(g.tensor())

⎡1  0       0     ⎤
⎢                 ⎥
⎢    2            ⎥
⎢0  φ       0     ⎥
⎢                 ⎥
⎢        2    2   ⎥
⎣0  0   φ ⋅sin (θ)⎦

In [3]:
V0, m, φ0, θ0, ψ0 = sympy.symbols("V0 m φ0 θ0 ψ0")
V = (V0 + 1/2 * m**2 * (φ - φ0)**2 - θ*θ0 - ψ*ψ0).nsimplify()
display(V)

In [4]:
hesse = SymbolicCalculation(coords, g, V).execute([[0,1,0],[0,0,1]])

Calculating orthonormal basis...


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Calculating covariant Hesse matrix...


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
out = Compiler(hesse, cleanup=False).compile()
out.print_sym_lookup_table()

[Symbol Dictionary]
φ -> x[0]
θ -> x[1]
ψ -> x[2]
V0 -> args[0]
m -> args[1]
φ0 -> args[2]
θ0 -> args[3]
ψ0 -> args[4]
